## Summary

Pandas! Pandas! Pandas!

### Dealing with 1 DataFrame

- 1.Data Transformation
     - lambda
     - map
     - apply


- 2.Data Aggregation & Group Operations
     - groupby
     - aggregate
     - transform
     - apply


- 3.Pivot Table & Cross Table
     - pivot_table
     - crosstab


### Dealing with 2+ DataFrames


- 4.Combining & Merging
     - concat
     - merge


In [1]:
import numpy as np
import pandas as pd

## Data Transformation

### lambda
- An anonymous function, a fancy way of defining a function.

In [2]:
l = lambda x:x**2
l(3)

9

In [3]:
def fun(x):
    return x**2
fun(3)

9

### map

- map values of Series using input correspondence (which can be a dict, Series, or function)

In [4]:
ser = pd.Series(['a','b','f'])
ser

0    a
1    b
2    f
dtype: object

In [5]:
ser.map({'a':'excellent','b':'fair','f':'failed'})

0    excellent
1         fair
2       failed
dtype: object

In [6]:
ser.map(lambda x:x.upper())

0    A
1    B
2    F
dtype: object

In [7]:
ser.map(str.upper)

0    A
1    B
2    F
dtype: object

### apply
- Apply a function along any axis of a DataFrame

In [8]:
frame = pd.DataFrame(np.arange(12).reshape(4,3), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [9]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    9
d    9
e    9
dtype: int64

In [10]:
frame.apply(f, axis = 1)

Utah      2
Ohio      2
Texas     2
Oregon    2
dtype: int64

In [11]:
frame.apply(lambda x:x.sum())

b    18
d    22
e    26
dtype: int64

In [12]:
frame.sum()

b    18
d    22
e    26
dtype: int64

## Data Aggregation & Group Operations
- groupby creates grouped objects
- aggregate, transform, apply act on grouped objects

### groupby
- Create a groupby object for futher use

In [13]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'], 'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.arange(5), 'data2' : np.random.randint(2,10,5)})
df

,data1,data2,key1,key2
0,0,4,a,one
1,1,6,a,two
2,2,4,b,one
3,3,4,b,two
4,4,3,a,one


- Group using own columns(s)

In [14]:
grouped = df.groupby(df['key1'])
grouped

In [15]:
df.groupby(['key1','key2']).sum()

data1  data2
key1 key2              
a    one       4      7
     two       1      6
b    one       2      4
     two       3      4

- Group using equal-length ndarray

In [16]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df.groupby([states,years]).mean()

data1  data2
California 2005    1.0    6.0
           2006    2.0    4.0
Ohio       2005    1.5    4.0
           2006    4.0    3.0

- Set arguments as_index = False to keep the columns

In [17]:
df.groupby(['key1','key2'], as_index = False).sum() 

,key1,key2,data1,data2
0,a,one,4,7
1,a,two,1,6
2,b,one,2,4
3,b,two,3,4


### aggregate
- Turn array into scalar value

In [18]:
df

,data1,data2,key1,key2
0,0,4,a,one
1,1,6,a,two
2,2,4,b,one
3,3,4,b,two
4,4,3,a,one


In [19]:
df.groupby('key1').max()

,data1,data2,key2
key1,,,
a,4,6,two
b,3,4,two


In [20]:
df.groupby('key1').sum()

,data1,data2
key1,,
a,5,13
b,5,8


In [21]:
g = lambda x:x.max() - x.min()
df.groupby('key1').agg(g)

,data1,data2
key1,,
a,4,3
b,1,0


In [22]:
def peak_bottom(x):
    return x.max() - x.min()
df.groupby('key1').agg(peak_bottom)

,data1,data2
key1,,
a,4,3
b,1,0


In [23]:
df.groupby('key1').agg([np.max, np.min, peak_bottom])

data1                  data2                 
      amax amin peak_bottom  amax amin peak_bottom
key1                                              
a        4    0           4     6    3           3
b        3    2           1     4    4           0

In [24]:
df.groupby('key1').agg({'data1': np.max, 'data2': np.min})

,data1,data2
key1,,
a,4,3
b,3,4


### transform
- Transform applies a function to each group, then places the result in the approciate locations. 

In [25]:
people = pd.DataFrame(np.random.randint(20, 50, 10).reshape(5,2), columns=['age', 'income'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,age,income
Joe,22,36
Steve,38,21
Wes,22,28
Jim,47,35
Travis,21,42


In [26]:
key = ['one', 'two', 'one', 'two', 'one']

In [27]:
people.groupby(key).agg(np.min)

,age,income
one,21,28
two,38,21


In [28]:
people.groupby(key).transform(np.min)

,age,income
Joe,21,28
Steve,38,21
Wes,21,28
Jim,38,21
Travis,21,28


In [29]:
def minus_min(x):
    return x - x.min()

In [30]:
people.groupby(key).transform(minus_min)

,age,income
Joe,1,8
Steve,0,0
Wes,1,0
Jim,9,14
Travis,0,14


### apply
- Split - Apply - Combine
- apply splits the object into pieces, invokes the passed function on each piece, then attempts to concatenate the pieces together.

In [31]:
tips = pd.read_csv('N3_tips.csv').dropna()
tips = tips[:10].reset_index(drop = True)
tips

,total_bill,tip,sex,smoker,day,time,size
0,12.90,1.10,Female,Yes,Sat,Dinner,2.0
1,11.87,1.63,Female,No,Thur,Lunch,2.0
2,3.07,1.00,Female,Yes,Sat,Dinner,1.0
3,24.08,2.92,Female,No,Thur,Lunch,4.0
4,18.71,4.00,Male,Yes,Thur,Lunch,3.0
5,24.55,2.00,Male,No,Sun,Dinner,4.0
6,13.94,3.06,Male,No,Sun,Dinner,2.0
7,10.33,2.00,Female,No,Thur,Lunch,2.0
8,18.29,3.76,Male,Yes,Sat,Dinner,4.0
9,8.77,2.00,Male,No,Sun,Dinner,2.0


In [49]:
def when_people_get_generous(df, n = 3):
    return df.sort_values(by = 'tip', ascending = False)[:n]

In [50]:
tips.groupby('time', as_index = False).apply(when_people_get_generous).reset_index(drop = True)

,total_bill,tip,sex,smoker,day,time,size
0,18.29,3.76,Male,Yes,Sat,Dinner,4.0
1,13.94,3.06,Male,No,Sun,Dinner,2.0
2,24.55,2.00,Male,No,Sun,Dinner,4.0
3,18.71,4.00,Male,Yes,Thur,Lunch,3.0
4,24.08,2.92,Female,No,Thur,Lunch,4.0
5,10.33,2.00,Female,No,Thur,Lunch,2.0


In [56]:
def who_are_generous(df, n = 2):
    return df.sort_values(by = 'total_bill', ascending = False)[:n]
tips.groupby('sex',as_index= False).apply(who_are_generous).reset_index(drop = True)

total_bill   tip     sex smoker   day    time  size
sex                                                          
Female 3       24.08  2.92  Female     No  Thur   Lunch   4.0
       0       12.90  1.10  Female    Yes   Sat  Dinner   2.0
Male   5       24.55  2.00    Male     No   Sun  Dinner   4.0
       4       18.71  4.00    Male    Yes  Thur   Lunch   3.0

## Pivot Table & Cross Table

### pivot_table

In [57]:
pt = pd.pivot_table(tips, values = 'total_bill', index = ['day', 'time'], 
                    columns = ['sex'], aggfunc = np.max)
pt

,sex,Female,Male
day,time,,
Sat,Dinner,12.90,18.29
Sun,Dinner,NaN,24.55
Thur,Lunch,24.08,18.71


In [36]:
pd.pivot_table(tips, values = 'tip', index = ['time'], columns = ['smoker'], aggfunc = 'count')

smoker,No,Yes
time,,
Dinner,3,3
Lunch,3,1


### crosstab
- A crosstab (cross-tabing) is a special case of a pivot table 
- By default, crosstab computes a frequency table of the factors 

In [37]:
pd.crosstab(tips.time, tips.smoker)

smoker,No,Yes
time,,
Dinner,3,3
Lunch,3,1


In [38]:
pd.crosstab(tips.time, tips.smoker, margins = True)

smoker,No,Yes,All
time,,,
Dinner,3,3,6
Lunch,3,1,4
All,6,4,10


## Combining & Merging

### concat

- Numpy - concatenate

In [39]:
arr = np.arange(6).reshape((2,3))
arr

array([[0, 1, 2],
       [3, 4, 5]])

In [40]:
np.concatenate([arr, arr], axis = 1)

array([[0, 1, 2, 0, 1, 2],
       [3, 4, 5, 3, 4, 5]])

- Pandas - concat

In [41]:
df = pd.DataFrame(arr)
df

,0,1,2
0,0,1,2
1,3,4,5


In [42]:
pd.concat([df, df])

,0,1,2
0,0,1,2
1,3,4,5
0,0,1,2
1,3,4,5


In [43]:
pd.concat([df, df], axis = 1)

,0,1,2,0,1,2
0,0,1,2,0,1,2
1,3,4,5,3,4,5


### merge

In [58]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a'], 'data1': range(5)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a


In [59]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [45]:
df_merge = pd.merge(df1, df2, on = 'key')
df_merge

,data1,key,data2
0,0,b,1
1,1,b,1
2,2,a,0
3,4,a,0


- By default, it's inner join; we can set argument how = 'left' to enable a left join

In [46]:
df_merge = pd.merge(df1, df2, how = 'left')
df_merge

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0


In [60]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a'],'data1': range(5)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a


In [61]:
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [63]:
df_merge = pd.merge(df3, df4, left_on='lkey', right_on='rkey')
df_merge.drop(2)

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
3,4,a,0,a
